In [2]:
import pandas as pd
df = pd.read_csv('wt.csv')

In [3]:
df.head()

,unitlocation,ttimestamplocal,windspeed,power,wind direction Angle,rtr_rpm,pitch Angle,generation,wheel hub temperature,ambient Temperature,Tower bottom ambient temperature,failure time
0,WTG40,\t2020-03-01 00:00:00\t,0.93,0.0,0.0,0.38,49.99,2872434,10.54,2.78,19.40,11.12
1,WTG40,\t2020-03-01 00:10:00\t,0.81,0.0,0.0,0.37,49.99,2872434,10.60,2.80,19.08,11.12
2,WTG40,\t2020-03-01 00:20:00\t,0.56,0.0,0.0,0.36,49.99,2872434,10.92,2.76,19.34,11.12
3,WTG40,\t2020-03-01 00:30:00\t,0.46,0.0,0.0,0.36,49.99,2872434,10.65,2.72,19.32,11.12
4,WTG40,\t2020-03-01 00:40:00\t,0.45,0.0,0.0,0.33,49.99,2872434,10.62,2.82,19.19,11.12


In [4]:
# remove spaces in columns name
df.columns = df.columns.str.replace(' ','_')

# splitting two WT in different dataset using location of turbine
grouped = df.groupby(df.unitlocation)
wtg40 = grouped.get_group("WTG40")

# get failure times
fault_time_wtg40 = wtg40.failure_time.unique()

# get the failure status in dataset
wtg40['total_runtime'] = [i*10 for i in range(1,len(wtg40)+1)]
wtg40['is_fault'] = wtg40['total_runtime']< fault_time_wtg40.max()

# remove some columns that are unnecessary
df = wtg40.drop(['unitlocation', 'ttimestamplocal','total_runtime', 'failure_time'], axis=1)

df['is_fault'] = df['is_fault'].astype('category')
df['is_fault'] = df['is_fault'].cat.codes

# final datasets for training
y = df['is_fault']
x= df.drop(['is_fault'],  axis=1)
x.describe()

/tmp/ipykernel_34708/3574493064.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wtg40['total_runtime'] = [i*10 for i in range(1,len(wtg40)+1)]
/tmp/ipykernel_34708/3574493064.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wtg40['is_fault'] = wtg40['total_runtime']< fault_time_wtg40.max()


,windspeed,power,wind_direction_Angle,rtr_rpm,pitch_Angle,generation,wheel_hub_temperature,ambient_Temperature,Tower_bottom_ambient_temperature
count,42592.000000,42592.000000,42592.000000,42592.000000,42592.000000,4.259200e+04,42592.000000,42592.000000,42592.000000
mean,4.693273,653.797555,171.081463,7.227109,8.868844,5.665854e+06,22.785580,15.840264,29.088985
std,2.770764,739.499557,48.802821,3.840320,21.170777,1.233734e+06,8.086884,9.347388,6.523025
min,0.020000,-29.840000,0.000000,0.210000,-0.580000,2.872434e+06,-4.830000,-13.060000,2.960000
25%,2.680000,60.047500,175.170000,5.000000,-0.520000,4.843144e+06,17.590000,9.950000,25.087500
50%,4.300000,323.215000,180.520000,7.120000,-0.500000,5.916913e+06,23.820000,17.280000,30.310000
75%,6.372500,1056.462500,185.160000,10.540000,1.950000,6.577421e+06,29.562500,23.180000,33.890000
max,19.210000,2224.870000,355.970000,12.480000,90.470000,7.529655e+06,37.210000,36.210000,44.830000


In [5]:
# check there is any null value present in dataset or not
print(x.isnull().values.any())
# 1 -> outlier  and 0-> normal
y

False


0        1
1        1
2        1
3        1
4        1
        ..
42587    0
42588    0
42589    0
42590    0
42591    0
Name: is_fault, Length: 42592, dtype: int8

In [23]:
y.value_counts()

0    42586
1        6
Name: is_fault, dtype: int64

In [18]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.33
x_train,X_test,y_train,Y_test = train_test_split(x,y,test_size=test_size,random_state=seed)

In [20]:
model = XGBClassifier()
eval_set = [(x_train,y_train),(X_test,Y_test)]
model.fit(x_train,y_train,early_stopping_rounds=10,eval_metric=["error","logloss"],eval_set=eval_set,verbose=True)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-error:0.00000	validation_0-logloss:0.43756	validation_1-error:0.00007	validation_1-logloss:0.43755
[1]	validation_0-error:0.00000	validation_0-logloss:0.29638	validation_1-error:0.00007	validation_1-logloss:0.29638
[2]	validation_0-error:0.00000	validation_0-logloss:0.20741	validation_1-error:0.00007	validation_1-logloss:0.20743
[3]	validation_0-error:0.00000	validation_0-logloss:0.14788	validation_1-error:0.00007	validation_1-logloss:0.14792
[4]	validation_0-error:0.00000	validation_0-logloss:0.10668	validation_1-error:0.00007	validation_1-logloss:0.10674
[5]	validation_0-error:0.00000	validation_0-logloss:0.07757	validation_1-error:0.00007	validation_1-logloss:0.07764
[6]	validation_0-error:0.00000	validation_0-logloss:0.05671	validation_1-error:0.00007	validation_1-logloss:0.05679
[7]	validation_0-error:0.00000	validation_0-logloss:0.04163	validation_1-error:0.00007	validation_1-logloss:0.04171
[8]	validation_0-error:0.00000	validation_0-logloss:0.03064	validation_1

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [21]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [22]:
#evaluate predictions
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy:  %.2f%%"%(accuracy *100.0))

Accuracy:  99.99%


In [14]:
from sklearn.ensemble import RandomForestClassifier
model2 = RandomForestClassifier()
model2.fit(x_train,y_train)

RandomForestClassifier()

In [15]:
y_pred2 = model2.predict(X_test)

In [17]:
#evaluate predictions
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_test, y_pred2)
print("Accuracy:  %.2f%%"%(accuracy *100.0))

Accuracy:  100.00%
